In [1]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
!pip install langchain transformers torch sentence-transformers chromadb


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.4 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-community # Install the required package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF langchain chromadb

import fitz  # PyMuPDF
import os
from google.colab import files
import shutil
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Path to the folder containing PDFs
pdf_folder_path = '/content/drive/MyDrive/ml algo books'

# Extract text from all PDFs in the folder
def extract_text_from_pdfs(folder_path):
    all_text = ""
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            document = fitz.open(pdf_path)
            for page_num in range(len(document)):
                page = document.load_page(page_num)
                all_text += page.get_text()
    return all_text

text = extract_text_from_pdfs(pdf_folder_path)
print("Text extracted from all PDFs.")

# Initialize the embedding model
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Initialize the Chroma vector store
vector_store = Chroma(
    persist_directory="./vector_store",
    embedding_function=embeddings
)

# Split the extracted text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=25)
texts = text_splitter.split_text(text)

# Add the text chunks to the vector store
vector_store.add_texts(texts)
vector_store.persist()

print("Vector store initialized and text added.")

# Zip the vector store directory
shutil.make_archive('vector_store', 'zip', './vector_store')

# Download the zipped vector store
files.download('vector_store.zip')


In [6]:
def retrieve_documents(query, k=3):
    # Use a valid search type
    search_type = 'similarity_score_threshold'
    results = vector_store.search(query, search_type=search_type, k=k)
    # Extract the 'page_content' attribute from Document objects
    return [result.page_content for result in results]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Initialize DistilGPT-2 model and tokenizer
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    # print("ignore",inputs["input_ids"])
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=300,early_stopping=True, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Initialize embeddings
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Load vector store
vector_store = Chroma(
    persist_directory=r"/content/drive/MyDrive/vectorDB/kpi_vectorDB",
    embedding_function=embeddings
)


In [ ]:
class RAGPipeline:
    def __init__(self, retriever, generator):
        self.retriever = retriever
        self.generator = generator

    def run(self, query):
        # Retrieve relevant documents
        docs = self.retriever(query)
        context = " ".join(docs)  # Concatenate retrieved documents
        # Generate response
        response = self.generator(context+'\n\n'+ query+'\n\n\n')
        return response

# Create the RAG pipeline
rag_pipeline = RAGPipeline(
    retriever=retrieve_documents,
    generator=generate_text
)

# Test with a query
query = """what are most important features if I run a shoe shop and i have data of daily visitors and the number of people who buy
          and the number of people who don't buy the shoe of a particular date , i also keep account of the price of product ,
          discount and new product arrivals so i want to find about which customer bought for which reason ?"""
response = rag_pipeline.run(query)



In [31]:
print('\n'.join(response.split('.')))

To fully understand what to increase or decrease, we need to look at the activities that created the sales (the result)
 Result indicators that lie beneath KRIs could include: Net proﬁt on key product lines 
Sales made yesterday might be “increased repeat business from key customers”? 
Surely you should be measuring the satisfaction of key customers reg- ularly and ignoring those customers you could do without  
Why converted it into a result indicator (e g , daily sales are a result of activities that have taken place to create the sales)  The KPI lies deeper down


In [26]:
ans=[]
for i in response.split('\n'):
  if len(i)==0:
    break
  elif len(i.split())>2:
    ans.append(i)

ans=" ".join(ans)
ans=list(set(ans.split('.')))
# print(ans)
if ans[-1]=='':
  ans='\n'.join(ans)
else:
  ans='\n'.join(ans[:len(ans)-1])

print(ans)

Sales made yesterday might be “increased repeat business from key customers”? Surely you should be measuring the satisfaction of key customers reg- ularly and ignoring those customers you could do without  Why converted it into a result indicator (e g , daily sales are a result of activities that have taken place to create the sales)  The KPI lies deeper down
